In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV
np.random.seed(0)

In [2]:

def scaler(scaler, data, test=None):
    scaler.fit(data)  # Apply transform to both the training set and the test set.
    train_scale = scaler.transform(data)
    if test is not None:
        test_scale = scaler.fit_transform(test)

    return train_scale, test_scale, scaler

def train_model(classifier, X_tr, y_tr, X_te, y_te):
    print('start training...')
    classifier.fit(X_tr, y_tr)
    print('evaluation...')
    y_p = classifier.predict(X_te)
    score = evaluate(y_te, y_p)
    print(f'score is {score}')
    return classifier, score

def evaluate(y_true, y_pred):
    return f1_score(y_true, y_pred, average="macro")

In [3]:
data = pd.read_table('data/dev.tsv')
test = pd.read_table('data/eval.tsv')

In [4]:
df = data.copy()
eval = test.copy()

In [59]:
from scipy import stats
def happy_sad(x):
    if x>df['valence'].mean():
        return 'happy'
    else:
        return 'sad'
df['boringness'] = df['loudness'] + df['tempo'] + (df['energy']*100) + (df['danceability']*100)
df['valence_happy_sad'] = df['valence'].apply(lambda x: happy_sad(x))
df['loudness_plus_60'] = df['loudness'].apply(lambda x: x+60)
df['loudness_pos'] = df['loudness'].apply(lambda x: -1*x)
df['loudness_pos'] = np.sqrt(df['loudness_pos'])
df['boringness_plus_60'] = df['boringness'].apply(lambda x: x+60)
df['duration_ms_box_cox_trans'] = stats.boxcox(df['duration_ms'])[0]
df['acousticness_sqrt_trans'] = np.sqrt(df['acousticness'])
df['liveness_sqrt_trans'] = np.sqrt(df['liveness'])
df['popularity_sqrt_trans'] = np.sqrt(df['popularity'])
df['speechiness_sqrt_trans'] = np.sqrt(df['speechiness'])

/home/omid/anaconda3/envs/ml/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [71]:
df = df.fillna(value=0)

In [103]:
col = [
    'valence',
        # 'year',
        # 'acousticness',
        # 'artists',
        # 'danceability',
       # 'duration_ms',
        'energy',
        'explicit',
        # 'id',
        'instrumentalness',
        'key',
       'liveness',
        # 'loudness',
        'popularity',
        # 'speechiness',
        'tempo',
       #  'mode',
       # 'loudness_plus_60',
        'loudness_pos',
         'boringness',
       #  'valence_happy_sad',
       # 'boringness_plus_60',
        'duration_ms_box_cox_trans',
       'acousticness_sqrt_trans',
       #  'liveness_sqrt_trans',
       # 'popularity_sqrt_trans',
        'speechiness_sqrt_trans'
        ]
X = df[col]
y = df['mode']

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)

In [105]:
from sklearn.preprocessing import *
X_train_scal, X_test_scal, x_scaler = scaler(RobustScaler(), X_train, X_test)


In [106]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


# clf = SVC(random_state=42)
# clf = GaussianNB()
# clf = MultinomialNB(alpha=10/100)
# clf = BernoulliNB(alpha=10/100)
# clf = LogisticRegression(penalty='l2',max_iter=10000,random_state=0)
clf = RandomForestClassifier(n_estimators=250, random_state=0)
# clf = SGDClassifier(loss='hinge', penalty='l2', validation_fraction=0.1,max_iter=100000,tol=0.00001)
# clf = KNeighborsClassifier(n_neighbors = 10)

score = train_model(clf,X_train_scal,y_train,X_test_scal,y_test)
print(f' score {score}')
# scores = cross_val_score(clf, X, y, cv=5,scoring='f1_macro')
# print(f'mean score {np.mean(scores)}, max score {np.max(scores)}, min score {np.min(scores)}')

start training...
evaluation...
score is 0.5924540918285437
 score (RandomForestClassifier(n_estimators=250, random_state=0), 0.5924540918285437)


In [93]:
print(f' score {score}')

 score (RandomForestClassifier(n_estimators=250, random_state=0), 0.5942895704992852)


In [94]:
from sklearn.neighbors import KNeighborsClassifier

score = []
for i in range(2,100,2):
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    sco = evaluate(y_test, y_pred)
    score.append(sco)
    print(f'{i} {sco:.4f}')

2 0.5199
4 0.5407
6 0.5485
8 0.5415
10 0.5321
12 0.5214
14 0.5174
16 0.5110
18 0.5043
20 0.4965
22 0.4898
24 0.4819
26 0.4787
28 0.4756
30 0.4692
32 0.4649
34 0.4637
36 0.4605
38 0.4578
40 0.4532
42 0.4505
44 0.4491
46 0.4486
48 0.4462
50 0.4429
52 0.4393
54 0.4369
56 0.4370
58 0.4353
60 0.4352
62 0.4343
64 0.4326
66 0.4324
68 0.4323
70 0.4316
72 0.4301
74 0.4287
76 0.4292
78 0.4291
80 0.4287
82 0.4277
84 0.4270
86 0.4270
88 0.4266
90 0.4269
92 0.4251
94 0.4244
96 0.4242
98 0.4229


In [49]:
def WriteOnFile(name, y_eval):
    f = open(name, "w")
    f.write("Id,Predicted\n")
    for index, i in enumerate(y_eval):
        f.write(f"{index},{i}\n")
    f.close


test = eval[col]
test_scal = x_scaler.transform(test)
y_pred = clf.predict(test_scal)
WriteOnFile('submission.csv',y_pred)

KeyError: "['duration_ms_box_cox_trans'] not in index"